# Triton on SageMaker - Ensemble + FIL Backend

## Set up Environment

Installs the dependencies required to package the model and run inferences using Triton server.

Also define the IAM role that will give SageMaker access to the model artifacts and the NVIDIA Triton ECR image.

In [ ]:
!pip install -qU pip awscli boto3 sagemaker 
!pip install nvidia-pyindex
!pip install tritonclient[http]

In [ ]:
import boto3
import json
import sagemaker
import time
import os
from sagemaker import get_execution_role
import pandas as pd
import numpy as np

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime")

In [ ]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [ ]:
region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise("UNSUPPORTED REGION")

In [ ]:
base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
triton_image_uri = "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.05-py3".format(
    account_id=account_id_map[region], region=region, base=base
)

## Package models and dependencies and uploading to S3

First we create the Triton config file for the XGBoost model being served by the FIL Backend.

In [ ]:
USE_GPU = True
FIL_MODEL_DIR = './model_repository/fil'

# Maximum size in bytes for input and output arrays
MAX_MEMORY_BYTES = 60_000_000
NUM_FEATURES = 15
NUM_CLASSES = 2
bytes_per_sample = (NUM_FEATURES + NUM_CLASSES) * 4
max_batch_size = MAX_MEMORY_BYTES // bytes_per_sample

IS_CLASSIFIER = True
model_format = 'xgboost_json'

# Select deployment hardware (GPU or CPU)
if USE_GPU:
    instance_kind = 'KIND_GPU'
else:
    instance_kind = 'KIND_CPU'

# whether the model is doing classification or regression    
if IS_CLASSIFIER:
    classifier_string = 'true'
else:
    classifier_string = 'false'

# whether to predict probabilites or not
predict_proba = False

if predict_proba:
    predict_proba_string = 'true'
else:
    predict_proba_string = 'false'

config_text = f"""backend: "fil"
max_batch_size: {max_batch_size}
input [                                 
 {{  
    name: "input__0"
    data_type: TYPE_FP32
    dims: [ {NUM_FEATURES} ]                    
  }} 
]
output [
 {{
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }}
]
instance_group [{{ kind: {instance_kind} }}]
parameters [
  {{
    key: "model_type"
    value: {{ string_value: "{model_format}" }}
  }},
  {{
    key: "predict_proba"
    value: {{ string_value: "{predict_proba_string}" }}
  }},
  {{
    key: "output_class"
    value: {{ string_value: "{classifier_string}" }}
  }},
  {{
    key: "threshold"
    value: {{ string_value: "0.5" }}
  }},
  {{
    key: "storage_type"
    value: {{ string_value: "AUTO" }}
  }}
]

dynamic_batching {{}}"""

config_path = os.path.join(FIL_MODEL_DIR, 'config.pbtxt')
with open(config_path, 'w') as file_:
    file_.write(config_text)

We follow the instructions [here](https://github.com/triton-inference-server/python_backend#2-packaging-the-conda-environment) for packaging preprocessing dependencies (here scikit-learn and pandas) to be used in the python backend as conda env file.

In [ ]:
!bash create_prep_env.sh
!cp preprocessing_env.tar.gz model_repository/preprocessing/

In [ ]:
# move label encoders into python preprocessing directory
!cp label_encoders.pkl model_repository/preprocessing/1/

In [ ]:
# move trained xgboost model into fil model directory
!mkdir -p model_repository/fil/1
!cp xgboost.json model_repository/fil/1/

In [ ]:
# create model version directory for ensemble model
!mkdir -p model_repository/ensemble/1

In [ ]:
!tar --exclude='.ipynb_checkpoints' -czvf model.tar.gz -C model_repository .

In [ ]:
model_uri = sagemaker_session.upload_data(path="model.tar.gz", key_prefix="triton-fil-ensemble")

## Create SageMaker Endpoint

We start off by creating a sagemaker model from the model files we uploaded to s3 in the previous step.

In this step we also provide an additional Environment Variable i.e. `SAGEMAKER_TRITON_DEFAULT_MODEL_NAME` which specifies the name of the model to be loaded by Triton. **The value of this key should match the folder name in the model package uploaded to s3.** This variable is optional in case of a single model. In case of ensemble models, this **key has to be specified** for Triton to startup in SageMaker.

Additionally, customers can set `SAGEMAKER_TRITON_BUFFER_MANAGER_THREAD_COUNT` and `SAGEMAKER_TRITON_THREAD_COUNT` for optimizing the thread counts.

In [ ]:
sm_model_name = "triton-fil-ensemble-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

container = {
    "Image": triton_image_uri,
    "ModelDataUrl": model_uri,
    "Environment": {"SAGEMAKER_TRITON_DEFAULT_MODEL_NAME": "ensemble"},
}

create_model_response = sm.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

Using the model above, we create an endpoint configuration where we can specify the type and number of instances we want in the endpoint.

In [ ]:
endpoint_config_name = "triton-fil-ensemble-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.4xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to InService once the deployment is successful.

In [ ]:
endpoint_name = "triton-fil-ensemble-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## Run Inference

Once we have the endpoint running we can use some sample raw data to do an inference using json as the payload format. For inference request format, Triton uses the KFServing community standard [inference protocols.](https://github.com/triton-inference-server/server/blob/main/docs/protocol/README.md)

In [ ]:
data_infer = pd.read_csv("data_infer.csv")
data_infer

In [ ]:
STR_COLUMNS = ['Time',
 'Amount',
 'Zip',
 'MCC',
 'Merchant Name',
 'Use Chip',
 'Merchant City',
 'Merchant State',
 'Errors?']

batch_size = len(data_infer)

payload = {}
payload["inputs"] = []
data_dict = {}
for col in data_infer.columns:
    data_dict[col] = {}
    data_dict[col]['name'] = col
    if col in STR_COLUMNS:
        data_dict[col]['data'] = data_infer[col].astype(str).tolist()
        data_dict[col]['datatype'] = 'BYTES'
    else:
        data_dict[col]['data'] = data_infer[col].astype('float32').tolist()
        data_dict[col]['datatype'] = 'FP32'
    data_dict[col]['shape'] = [batch_size, 1]
    payload["inputs"].append(data_dict[col])

In [ ]:
payload

In [ ]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/octet-stream", Body=json.dumps(payload)
)

In [ ]:
response_body = json.loads(response["Body"].read().decode("utf8"))
predictions = response_body['outputs'][0]['data']

In [ ]:
CLASS_LABELS = ['NOT FRAUD', 'FRAUD']
predictions = [CLASS_LABELS[int(idx)] for idx in predictions]

In [ ]:
print(predictions)

## Terminate endpoint and clean up artifacts

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_model(ModelName=sm_model_name)